# In-class exercise: turbulence simulations analysis


1. Choose file # 10 and normalise quantities to CGS units.


2. Make a plot of density with superimposed velocity vectors.


3. Make a plot of thermal pressure with superimposed magnetic field vectors.


Remember that the thermal pressure for isothermal gas is:

$p = \rho\,c_{\rm iso}^2$, where $c_{\rm iso}$ is the isothermal sound speed (see **units.out**).

To normalise the magnetic field, we can use the following normalisation factor:

$B_0 = v_0\,\sqrt{4\,\pi\,\rho_0}$


4. The times can be retrieved from the second column of the **vtk.out** file in code units. To normalise it to CGS units, we can first calculate the normalisation factor for time:

$t_0 = \frac{L_0}{v_0}$


5. Make a histogram of the density field and fit an appropriate function using regression methods.


6. Create a function that calculates the following quantities and prints them in a pandas row with time in the first column and headers:


 - density average 
 
 - density standard deviation
 
 - thermal pressure average
 
 - thermal pressure standard deviation

### Import libraries:

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv

In [6]:
import pandas as pd
from scipy.optimize import curve_fit

### Read mesh:

In [7]:
mesh = pv.read("./TURB_DRIVE_SUP/data.0010.vtk")

In [8]:
print(mesh)

RectilinearGrid (0x11f842d00)
  N Cells:	2500
  N Points:	2601
  X Bounds:	-2.000e+00, 2.000e+00
  Y Bounds:	-2.000e+00, 2.000e+00
  Z Bounds:	0.000e+00, 0.000e+00
  Dimensions:	51, 51, 1
  N Arrays:	5



### Get data arrays in code units:

In [11]:
rho = pv.get_array(mesh, "rho", preference = 'cell')
vx1 = pv.get_array(mesh, "vx1", preference = 'cell')
vx2 = pv.get_array(mesh, "vx2", preference = 'cell')
Bx1 = pv.get_array(mesh, "Bx1", preference = 'cell')
Bx2 = pv.get_array(mesh, "Bx2", preference = 'cell')

In [12]:
print(rho.shape)

(2500,)


### Normalisation:

Remember that we need to read the **units.out** file and multiply the arrays above by the normalisation constants:

In [13]:
data = pd.read_csv("./TURB_DRIVE_SUP/units.out")

In [14]:
print(data)

  variable  normalisation   units
0    rho_0   1.660000e-24  g/cm^3
1      v_0   1.000000e+06    cm/s
2      L_0   3.086000e+18      cm
3    c_iso   1.000000e+06    cm/s


In [15]:
rho_0   = np.array(data.loc[data["variable"] == "rho_0"]["normalisation"])
vel_0   = np.array(data.loc[data["variable"] == "v_0"]["normalisation"])
len_0   = np.array(data.loc[data["variable"] == "L_0"]["normalisation"])
c_iso   = np.array(data.loc[data["variable"] == "c_iso"]["normalisation"])

### New normalisations:

In [17]:
tim_0 = (len_0/vel_0)

print(tim_0) # in seconds

[3.086e+12]


Magnetic field:
$B_0 = v_0\,\sqrt{4\,\pi\,\rho_0}$

In [20]:
bfi_0 = vel_0*np.sqrt(4*np.pi*rho_0)

print(bfi_0) # in Gauss

[4.56729408e-06]


### Normalise:

In [21]:
rho_cgs = rho*rho_0

vx1_cgs = vx1*vel_0
vx2_cgs = vx2*vel_0

Bx1_cgs = Bx1*bfi_0
Bx2_cgs = Bx2*bfi_0

In [24]:
print(Bx1_cgs.shape)

(2500,)


### Convert to 2D:

In [25]:
rho_cgs2D = rho_cgs.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)

vx1_cgs2D = vx1_cgs.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)
vx2_cgs2D = vx2_cgs.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)

Bx1_cgs2D = Bx1_cgs.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)
Bx2_cgs2D = Bx2_cgs.reshape(mesh.dimensions[0] - 1, mesh.dimensions[1] - 1)